In [5]:
from datasets import load_dataset

dataset = load_dataset("allenai/ai2_arc", "ARC-Easy")
dataset["train"][0]

{'id': 'Mercury_7220990',
 'question': 'Which factor will most likely cause a person to develop a fever?',
 'choices': {'text': ['a leg muscle relaxing after exercise',
   'a bacterial population in the bloodstream',
   'several viral particles on the skin',
   'carbohydrates being digested in the stomach'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'B'}

In [7]:
# print names of files in directory "LogicBench/LogicBench(Eval)"
import os
path = "LogicBench/LogicBench(Eval)"
files = os.listdir(path)
files


['BQA', 'MCQA', 'Readme.md']